In [13]:
'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 10000
nb_validation_samples = 2500
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=nb_train_samples,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=nb_validation_samples,
    class_mode='binary')

x_train, y_train = next(train_generator)
x_val, y_val = next(validation_generator)

model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_val, y_val))

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Train on 10000 samples, validate on 2500 samples
Epoch 1/50
10000/10000 [==============================] - 22s 2ms/step - loss: 0.6707 - acc: 0.6120 - val_loss: 0.5623 - val_acc: 0.7048
Epoch 2/50
10000/10000 [==============================] - 21s 2ms/step - loss: 0.5827 - acc: 0.7041 - val_loss: 0.5204 - val_acc: 0.7456
Epoch 3/50
10000/10000 [==============================] - 21s 2ms/step - loss: 0.5350 - acc: 0.7422 - val_loss: 0.4952 - val_acc: 0.7724
Epoch 4/50
10000/10000 [==============================] - 22s 2ms/step - loss: 0.5062 - acc: 0.7746 - val_loss: 0.6758 - val_acc: 0.7256
Epoch 5/50
10000/10000 [==============================] - 22s 2ms/step - loss: 0.4867 - acc: 0.7826 - val_loss: 0.5324 - val_acc: 0.7432
Epoch 6/50
10000/10000 [==============================] - 22s 2ms/step - loss: 0.4651 - acc: 0.7897 - val_loss: 0.4803 - val_acc: 0.7720
Epoch 7/50
10000/10000 [====================

In [16]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=nb_train_samples,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=nb_validation_samples,
    class_mode='binary')

from timeit import default_timer as timer

for e in range(epochs):
    print('Epoch', e)
    start = timer()
    x_train, y_train = next(train_generator)
    x_val, y_val = next(validation_generator)
    # batches = 0
    # for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=32):
    mid = timer()
    print('image_processing: {}'.format(mid-start))
    model.fit(x_train, y_train, batch_size=batch_size, validation_data=(x_val, y_val))
    # batches += 1
    # if batches >= len(x_train) / 32:
    #     # we need to break the loop by hand because
    #     # the generator loops indefinitely
    #     break
    end = timer()
    print('model_fitting: {}'.format(end-mid))

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
('Epoch', 0)
image_processing: 82.0544009209
Train on 10000 samples, validate on 2500 samples
Epoch 1/1
10000/10000 [==============================] - 21s 2ms/step - loss: 0.5743 - acc: 0.7456 - val_loss: 0.4605 - val_acc: 0.8076
model_fitting: 21.4903039932
('Epoch', 1)
image_processing: 82.2597548962
Train on 10000 samples, validate on 2500 samples
Epoch 1/1
10000/10000 [==============================] - 21s 2ms/step - loss: 0.5389 - acc: 0.7554 - val_loss: 0.4679 - val_acc: 0.8116
model_fitting: 21.3692400455
('Epoch', 2)
image_processing: 82.5517599583
Train on 10000 samples, validate on 2500 samples
Epoch 1/1
10000/10000 [==============================] - 22s 2ms/step - loss: 0.5345 - acc: 0.7637 - val_loss: 0.4864 - val_acc: 0.7828
model_fitting: 21.5315899849
('Epoch', 3)
image_processing: 83.1872169971
Train on 10000 samples, validate on 2500 samples
Epoch 1/1
10000/10000 [=====================

KeyboardInterrupt: 

In [15]:
model.save_weights('first_try.h5')